# Testando a funcinalidade da classe MongoDBGraphStore

Tutorial desenvolvido com base em:
- [GraphRAG com MongoDB e LangChain](https://www.mongodb.com/pt-br/docs/atlas/atlas-vector-search/ai-integrations/langchain/graph-rag/)

Materiais úteis:
- [MongoDB as a Graph Database](https://www.mongodb.com/resources/basics/databases/mongodb-graph-database)
- [MongoDBGraphStore](https://langchain-mongodb.readthedocs.io/en/latest/langchain_mongodb/graphrag/langchain_mongodb.graphrag.graph.MongoDBGraphStore.html)
- [How to construct knowledge graphs](https://python.langchain.com/docs/how_to/graph_constructing/)
- [Graph RAG](https://python.langchain.com/docs/integrations/retrievers/graph_rag/)


## Imports e carregamento de variáveis de ambiente

In [1]:
import os
from src.settings import BASE_DIR, SQLITE_DB_NAME, MONGO_URI, MONGO_DB_NAME, MONGO_COLLECTION

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## Criando o agente de recuperação de informação utilizando o GraphRAG

O agente consulta um banco de dados relacional por meio de aprendizado por contexto armazenado em um banco de dados vetorial

In [2]:
# Inicialize o modelo de idioma

from langchain_openai import OpenAI
from langchain.chat_models import init_chat_model

# model = "gpt-4o"
model="gpt-4o-mini"
chat_model = init_chat_model(model, model_provider="openai", temperature=0)

In [3]:
# Carregue os dados de amostra

from langchain_community.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter

wikipedia_pages = WikipediaLoader(query="Sherlock Holmes", load_max_docs=3).load()

text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=0)
wikipedia_docs = text_splitter.split_documents(wikipedia_pages)

In [4]:
wikipedia_pages

[Document(metadata={'title': 'Sherlock Holmes', 'summary': 'Sherlock Holmes () is a fictional detective created by British author Arthur Conan Doyle. Referring to himself as a "consulting detective" in his stories, Holmes is known for his proficiency with observation, deduction, forensic science and logical reasoning that borders on the fantastic, which he employs when investigating cases for a wide variety of clients, including Scotland Yard.\nThe character Sherlock Holmes first appeared in print in 1887\'s A Study in Scarlet. His popularity became widespread with the first series of short stories in The Strand Magazine, beginning with "A Scandal in Bohemia" in 1891; additional tales appeared from then until 1927, eventually totalling four novels and 56 short stories. All but one are set in the Victorian or Edwardian eras between 1880 and 1914. Most are narrated by the character of Holmes\'s friend and biographer, Dr. John H. Watson, who usually accompanies Holmes during his investiga

In [5]:
wikipedia_docs

[Document(metadata={'title': 'Sherlock Holmes', 'summary': 'Sherlock Holmes () is a fictional detective created by British author Arthur Conan Doyle. Referring to himself as a "consulting detective" in his stories, Holmes is known for his proficiency with observation, deduction, forensic science and logical reasoning that borders on the fantastic, which he employs when investigating cases for a wide variety of clients, including Scotland Yard.\nThe character Sherlock Holmes first appeared in print in 1887\'s A Study in Scarlet. His popularity became widespread with the first series of short stories in The Strand Magazine, beginning with "A Scandal in Bohemia" in 1891; additional tales appeared from then until 1927, eventually totalling four novels and 56 short stories. All but one are set in the Victorian or Edwardian eras between 1880 and 1914. Most are narrated by the character of Holmes\'s friend and biographer, Dr. John H. Watson, who usually accompanies Holmes during his investiga

In [6]:
wikipedia_docs[0].metadata.keys()

dict_keys(['title', 'summary', 'source'])

In [7]:
wikipedia_docs[0].page_content

'Sherlock Holmes () is a fictional detective created by British author Arthur Conan Doyle. Referring to himself as a "consulting detective" in his stories, Holmes is known for his proficiency with observation, deduction, forensic science and logical reasoning that borders on the fantastic, which he employs when investigating cases for a wide variety of clients, including Scotland Yard.\nThe character Sherlock Holmes first appeared in print in 1887\'s A Study in Scarlet. His popularity became widespread with the first series of short stories in The Strand Magazine, beginning with "A Scandal in Bohemia" in 1891; additional tales appeared from then until 1927, eventually totalling four novels and 56 short stories. All but one are set in the Victorian or Edwardian eras between 1880 and 1914. Most are narrated by the character of Holmes\'s friend and biographer, Dr. John H. Watson, who usually accompanies Holmes during his investigations and often shares quarters with him at the address of 

In [9]:
# Instancie o armazenamento de gráficos

from langchain_mongodb.graphrag.graph import MongoDBGraphStore
from pymongo.errors import CollectionInvalid

try:
    graph_store = MongoDBGraphStore.from_connection_string(
        connection_string=MONGO_URI,
        database_name=MONGO_DB_NAME,
        collection_name=MONGO_COLLECTION,
        entity_extraction_model=chat_model
    )
except  CollectionInvalid:
    graph_store = MongoDBGraphStore(
        connection_string=MONGO_URI,
        database_name=MONGO_DB_NAME,
        collection_name=MONGO_COLLECTION,
        entity_extraction_model=chat_model
    )

In [10]:
graph_store.add_documents(wikipedia_docs)

Targets, types, and attributes do not have the same length for Sherlock Holmes!
Targets, types, and attributes do not have the same length for Dr. John H. Watson!
Targets, types, and attributes do not have the same length for Edgar Allan Poe!
Targets, types, and attributes do not have the same length for C. Auguste Dupin!
Targets, types, and attributes do not have the same length for Émile Gaboriau!
Targets, types, and attributes do not have the same length for Joseph Bell!
Targets, types, and attributes do not have the same length for Sir Henry Littlejohn!
Targets, types, and attributes do not have the same length for Sherlock Holmes!
Targets, types, and attributes do not have the same length for Dr. John Watson!
Targets, types, and attributes do not have the same length for Irene Adler!
Targets, types, and attributes do not have the same length for Lord Henry Blackwood!
Targets, types, and attributes do not have the same length for Sherlock Holmes: A Game of Shadows!
Targets, types, 

[BulkWriteResult({'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 8, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': [{'index': 0, '_id': 'Sherlock Holmes'}, {'index': 1, '_id': 'Arthur Conan Doyle'}, {'index': 2, '_id': 'Dr. John H. Watson'}, {'index': 3, '_id': 'Edgar Allan Poe'}, {'index': 4, '_id': 'C. Auguste Dupin'}, {'index': 5, '_id': 'Émile Gaboriau'}, {'index': 6, '_id': 'Joseph Bell'}, {'index': 7, '_id': 'Sir Henry Littlejohn'}]}, acknowledged=True),
 BulkWriteResult({'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 13, 'nMatched': 1, 'nModified': 1, 'nRemoved': 0, 'upserted': [{'index': 1, '_id': 'Dr. John Watson'}, {'index': 2, '_id': 'Irene Adler'}, {'index': 3, '_id': 'Lord Henry Blackwood'}, {'index': 4, '_id': 'Mary Morstan'}, {'index': 5, '_id': 'Guy Ritchie'}, {'index': 6, '_id': 'Joel Silver'}, {'index': 7, '_id': 'Lionel Wigram'}, {'index': 8, '_id': 'Susan Downey'}, {'index': 9, '_id': 'Dan Lin'}, {'ind

In [11]:
# Responda a perguntas sobre seus dados

query = "Who inspired Sherlock Holmes?"
answer = graph_store.chat_response(query)
print(answer.content)

Sherlock Holmes, the iconic detective created by Sir Arthur Conan Doyle, was inspired by Dr. Joseph Bell, one of his professors at the University of Edinburgh Medical School. Dr. Bell was known for his keen powers of observation and deduction, which he applied in diagnosing patients. Conan Doyle admired Bell's analytical skills and incorporated these traits into the character of Holmes.


In [12]:
# Responda a perguntas sobre seus dados

query = "Quantos filmes foram lançados em 2024?"
answer = graph_store.chat_response(query)
print(answer.content)

Não há informações disponíveis sobre filmes lançados em 2024.


Integrando com o GraphRAGRetriever

In [13]:
from langchain_mongodb.retrievers.graphrag import MongoDBGraphRAGRetriever

retriever = MongoDBGraphRAGRetriever(graph_store=graph_store)

In [24]:
# query = "Quem foi Sherlock Holmes?"
query = "Quem inspirou Sherlock Holmes?"
# resultados = retriever.get_relevant_documents(query=query, run_manager=None)
resultados = retriever.invoke(input=query)

# Exibir os resultados
for doc in resultados:
    print(doc.page_content)

{'_id': 'Sherlock Holmes', 'type': 'Character', 'attributes': {'created_by': ['Arthur Conan Doyle', 'Sir Arthur Conan Doyle'], 'description': ['fictional detective', 'consulting detective'], 'first_appearance': ['1887'], 'notable_works': ['A Study in Scarlet', 'The Strand Magazine'], 'setting': ['Victorian era', 'Edwardian era', '1890'], 'total_stories': ['4 novels', '56 short stories'], 'portrayed_by': ['Robert Downey Jr.'], 'role': ['Detective']}, 'relationships': {'attributes': [{}, {}, {}, {}, {}], 'target_ids': [], 'types': ['Accompanied By', 'Client', 'Partner', 'Adversary', 'Adversary']}}


In [25]:
resultados

[Document(metadata={}, page_content="{'_id': 'Sherlock Holmes', 'type': 'Character', 'attributes': {'created_by': ['Arthur Conan Doyle', 'Sir Arthur Conan Doyle'], 'description': ['fictional detective', 'consulting detective'], 'first_appearance': ['1887'], 'notable_works': ['A Study in Scarlet', 'The Strand Magazine'], 'setting': ['Victorian era', 'Edwardian era', '1890'], 'total_stories': ['4 novels', '56 short stories'], 'portrayed_by': ['Robert Downey Jr.'], 'role': ['Detective']}, 'relationships': {'attributes': [{}, {}, {}, {}, {}], 'target_ids': [], 'types': ['Accompanied By', 'Client', 'Partner', 'Adversary', 'Adversary']}}")]

In [ ]:
from typing import List, TypedDict
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langgraph.graph import StateGraph
from langchain.tools import StructuredTool
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_mongodb.retrievers.graphrag import MongoDBGraphRAGRetriever
from langchain_mongodb.graphrag.graph import MongoDBGraphStore
from pymongo.errors import CollectionInvalid

# -------------------------------
# 1. Estado do grafo
# -------------------------------
class MyState(TypedDict):
    messages: List[BaseMessage]

# -------------------------------
# 2. Modelo LLM
# -------------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente que responde perguntas com base em um grafo de conhecimento."),
    ("user", "{input}")
])

chain = LLMChain(llm=llm, prompt=prompt)

# -------------------------------
# 3. Retriever + StructuredTool
# -------------------------------
retriever = MongoDBGraphRAGRetriever(
    graph_store=graph_store
)

class ConsultaInput(BaseModel):
    consulta: str

def consultar_grafo(consulta: str) -> str:
    """
    Consulta um grafo de conhecimento com uma string de busca.
    """
    docs = retriever.invoke(consulta)
    if not docs:
        return "Nenhum resultado encontrado no grafo."
    return "\n\n".join([doc.page_content for doc in docs])

tool_consulta = StructuredTool.from_function(
    func=consultar_grafo,
    name="consultar_grafo",
    description="Consulta o grafo com base em uma pergunta em linguagem natural.",
    args_schema=ConsultaInput
)

# -------------------------------
# 4. Nó: interpretar com LLM
# -------------------------------
def interpretar(state: MyState) -> MyState:
    """
    Usa o modelo LLM para reescrever ou interpretar a pergunta do usuário.
    """
    ultima = state["messages"][-1]
    if not isinstance(ultima, HumanMessage):
        raise ValueError("A última mensagem deve ser do tipo HumanMessage.")
    
    resposta = chain.invoke({"input": ultima.content})
    nova_mensagem = AIMessage(content=resposta["text"])
    return {"messages": state["messages"] + [nova_mensagem]}

# -------------------------------
# 5. Nó: executar StructuredTool e retornar AIMessage
# -------------------------------
def consultar_grafo_node(state: MyState) -> MyState:
    """
    Invoca a StructuredTool e adiciona a resposta como uma AIMessage.
    """
    ultima = state["messages"][-1]
    print(ultima)
    if not isinstance(ultima, AIMessage):
        raise ValueError("Esperava-se uma AIMessage da etapa anterior.")

    resultado = tool_consulta.invoke({"consulta": ultima.content})
    nova_mensagem = AIMessage(content=resultado)
    return {"messages": state["messages"] + [nova_mensagem]}

# -------------------------------
# 6. Nó: formatar resposta final
# -------------------------------
def formatar_resposta(state: MyState) -> MyState:
    """
    Retorna apenas a última mensagem do estado como resposta final.
    """
    if not state["messages"]:
        raise ValueError("Nenhuma mensagem encontrada no estado.")
    return {"messages": [state["messages"][-1]]}

# -------------------------------
# 7. Criar o grafo
# -------------------------------
workflow = StateGraph(state_schema=MyState)
workflow.add_node("interpretar", interpretar)
# workflow.add_node("consultar_grafo", consultar_grafo_node)
workflow.add_node("formatar_resposta", formatar_resposta)

workflow.set_entry_point("interpretar")
# workflow.add_edge("interpretar", "consultar_grafo")
workflow.add_edge("interpretar", "formatar_resposta")

graph = workflow.compile()

# -------------------------------
# 8. Executar
# -------------------------------
estado_inicial = {
    "messages": [HumanMessage(content="Quem foi Sherlock Holmes?")]
}

resposta = graph.invoke(estado_inicial)

# -------------------------------
# 9. Mostrar resultado final
# -------------------------------
final = resposta["messages"][-1].content
print("📘 Resposta final:\n", final)


📘 Resposta final:
 Sherlock Holmes é um personagem fictício criado pelo autor britânico Sir Arthur Conan Doyle. Ele é um detetive consultor conhecido por suas habilidades de observação, raciocínio lógico e dedução. Holmes apareceu pela primeira vez na novela "A Estrela de Pérsia" (1887) e se tornou um dos personagens mais icônicos da literatura policial. Ele é frequentemente acompanhado por seu amigo e narrador das histórias, Dr. John Watson. As aventuras de Holmes se passam principalmente em Londres, e ele é famoso por resolver casos complexos que desafiam a polícia. O personagem inspirou inúmeras adaptações em filmes, séries de televisão e outras mídias.


In [63]:
resposta

{'messages': [AIMessage(content='Sherlock Holmes é um personagem fictício criado pelo autor britânico Sir Arthur Conan Doyle. Ele é um detetive consultor conhecido por suas habilidades de observação, raciocínio lógico e dedução. Holmes apareceu pela primeira vez na novela "A Estrela de Pérsia" (1887) e se tornou um dos personagens mais icônicos da literatura policial. Ele é frequentemente acompanhado por seu amigo e narrador das histórias, Dr. John Watson. As aventuras de Holmes se passam principalmente em Londres, e ele é famoso por resolver casos complexos que desafiam a polícia. O personagem inspirou inúmeras adaptações em filmes, séries de televisão e outras mídias.', additional_kwargs={}, response_metadata={})]}

In [ ]:
from typing import List, TypedDict
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langgraph.graph import StateGraph
from langchain.tools import StructuredTool
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_mongodb.retrievers.graphrag import MongoDBGraphRAGRetriever
from langchain_mongodb.graphrag.graph import MongoDBGraphStore
from pymongo.errors import CollectionInvalid
from langchain_openai import OpenAI
from langchain.chat_models import init_chat_model




# -------------------------------
# 1. Estado do grafo
# -------------------------------
class MyState(TypedDict):
    messages: List[BaseMessage]


# -------------------------------
# 2. Tool estruturada: consulta ao grafo
# -------------------------------
class QueryInput(BaseModel):
    query: str


class KnowledgeTool:
    def __init__(self, retriever: MongoDBGraphRAGRetriever):
        self.tool = StructuredTool.from_function(
            func=self._search,
            name="graph_search",
            description="Consulta o grafo com base em uma question em linguagem natural.",
            args_schema=QueryInput
        )
        self.retriever = retriever

    def _search(self, query: str) -> str:
        """
        Consulta um grafo de conhecimento com uma string de busca.
        """
        docs = self.retriever.invoke(query)
        if not docs:
            return "Nenhum resultado encontrado no grafo."
        return "\n\n".join([doc.page_content for doc in docs])

    def run(self, state: MyState) -> MyState:
        last_msg = state["messages"][-1]
        if not isinstance(last_msg, AIMessage):
            raise ValueError("Esperava-se uma AIMessage da etapa anterior.")

        resultado = self.tool.invoke({"query": last_msg.content})
        nova = AIMessage(content=resultado)
        return {"messages": state["messages"] + [nova]}


# -------------------------------
# 3. Agente conversacional com grafo
# -------------------------------
class ConversationalAgent:
    def __init__(self, retriever: MongoDBGraphRAGRetriever, model="gpt-4o-mini", temperature=0):
        self.llm = ChatOpenAI(model=model, temperature=temperature)
        self.chain = LLMChain(
            llm=self.llm,
            prompt=ChatPromptTemplate.from_messages([
                ("system", "Você é um assistente que responde questions com base em um grafo de conhecimento."),
                ("user", "{input}")
            ])
        )
        self.knowledge_tool = KnowledgeTool(retriever)
        self.graph = self._graph_create()

    def _interpret(self, state: MyState) -> MyState:
        """
        Usa o modelo LLM para reescrever ou interpretar a question do usuário.
        """
        last_msg = state["messages"][-1]
        if not isinstance(last_msg, HumanMessage):
            raise ValueError("Esperava-se uma HumanMessage.")
        
        response = self.chain.invoke({"input": last_msg.content})
        nova = AIMessage(content=response["text"])
        return {"messages": state["messages"] + [nova]}

    def _format_response(self, state: MyState) -> MyState:
        """
        Retorna apenas a última mensagem como resposta final.
        """
        return {"messages": [state["messages"][-1]]}

    def _graph_create(self):
        workflow = StateGraph(state_schema=MyState)
        workflow.add_node("interpret", self._interpret)
        workflow.add_node("graph_search", self.knowledge_tool.run)
        workflow.add_node("format_response", self._format_response)

        workflow.set_entry_point("interpret")
        # workflow.add_edge("interpretar", "graph_search")
        # workflow.add_edge("graph_search", "format_response")
        workflow.add_edge("interpret", "format_response")

        return workflow.compile()

    def response(self, questions: List[str]):
        """
        Processa múltiplas perguntas mantendo o histórico da conversa.
        """
        history: List[BaseMessage] = []

        for question in questions:
            print(f"\nPergunta: {question}")
            history.append(HumanMessage(content=question))
            state = {"messages": history}

            response = self.graph.invoke(state)
            response_msg = response["messages"][-1]
            history.append(response_msg)

            print("Resposta:", response_msg.content)


# -------------------------------
# 4. Execução: criar agente e interagir
# -------------------------------
if __name__ == "__main__":
    # model = "gpt-4o"
    model="gpt-4o-mini"
    chat_model = init_chat_model(model, model_provider="openai", temperature=0)

    # Instancie o armazenamento de gráficos
    try:
        graph_store = MongoDBGraphStore.from_connection_string(
            connection_string=MONGO_URI,
            database_name=MONGO_DB_NAME,
            collection_name=MONGO_COLLECTION,
            entity_extraction_model=chat_model
        )
    except  CollectionInvalid:
        graph_store = MongoDBGraphStore(
            connection_string=MONGO_URI,
            database_name=MONGO_DB_NAME,
            collection_name=MONGO_COLLECTION,
            entity_extraction_model=chat_model
        )

    # Substitua com seu retriever
    retriever = MongoDBGraphRAGRetriever(
        graph_store=graph_store
    )

    agent = ConversationalAgent(retriever, model="gpt-4o-mini")
    questions = [
        "Quem foi Sherlock Holmes?",
        # "Qual era a profissão do Dr. Watson?",
        # "Onde Sherlock morava?"
    ]
    agent.response(questions)



Pergunta: Quem foi Sherlock Holmes?
Resposta: Sherlock Holmes é um personagem fictício criado pelo autor britânico Sir Arthur Conan Doyle. Ele é um detetive consultor conhecido por suas habilidades de observação, raciocínio lógico e dedução. Holmes apareceu pela primeira vez na novela "A Study in Scarlet" (Um Estudo em Vermelho), publicada em 1887. Ao longo dos anos, ele se tornou um dos detetives mais icônicos da literatura, protagonizando várias histórias, incluindo "The Sign of the Four" (O Signo dos Quatro) e "The Hound of the Baskervilles" (O Cão dos Baskerville).

Holmes é frequentemente acompanhado por seu amigo e narrador das histórias, Dr. John Watson. O personagem é conhecido por seu uso de métodos científicos e sua capacidade de resolver mistérios complexos, muitas vezes desafiando a polícia. A figura de Sherlock Holmes teve um impacto duradouro na cultura popular e inspirou inúmeras adaptações em filmes, séries de televisão e outras mídias.


In [4]:
from typing import List
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.tools import StructuredTool
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_mongodb.retrievers.graphrag import MongoDBGraphRAGRetriever
from langchain_mongodb.graphrag.graph import MongoDBGraphStore
from pymongo.errors import CollectionInvalid
from langchain.chat_models import init_chat_model


# -------------------------------
# 1. Tool estruturada: consulta ao grafo
# -------------------------------
class QueryInput(BaseModel):
    query: str


class KnowledgeTool:
    def __init__(self, retriever: MongoDBGraphRAGRetriever):
        self.tool = StructuredTool.from_function(
            func=self._search,
            name="graph_search",
            description="Consulta o grafo com base em uma pergunta em linguagem natural.",
            args_schema=QueryInput
        )
        self.retriever = retriever

    def _search(self, query: str) -> str:
        docs = self.retriever.invoke(query)
        if not docs:
            return "Nenhum resultado encontrado no grafo."
        return "\n\n".join([doc.page_content for doc in docs])


# -------------------------------
# 2. Agente conversacional (sem langgraph)
# -------------------------------
class ConversationalAgentManual:
    def __init__(self, retriever: MongoDBGraphRAGRetriever, model="gpt-4o-mini", temperature=0):
        self.llm = ChatOpenAI(model=model, temperature=temperature)
        self.chain = LLMChain(
            llm=self.llm,
            prompt=ChatPromptTemplate.from_messages([
                ("system", "Você é um assistente que responde perguntas com base em um grafo de conhecimento."),
                ("user", "{input}")
            ])
        )
        self.knowledge_tool = KnowledgeTool(retriever)

    def interpret(self, message: HumanMessage) -> AIMessage:
        print('interpret message', message)
        response = self.chain.invoke({"input": message.content})
        print('interpret response', response)
        return AIMessage(content=response["text"])

    def graph_search(self, message: AIMessage) -> AIMessage:
        print('graph_search message', message)
        result = self.knowledge_tool.tool.invoke({"query": message.content})
        print('graph_search result ', result)
        return AIMessage(content=result)

    def response(self, questions: List[str]):
        history: List[BaseMessage] = []

        for question in questions:
            print(f"\nPergunta: {question}")
            user_msg = HumanMessage(content=question)
            history.append(user_msg)

            # Etapa de interpretação com LLM
            interpreted_msg = self.interpret(user_msg)
            print(interpreted_msg)
            history.append(interpreted_msg)

            # Etapa de busca no grafo
            result_msg = self.graph_search(interpreted_msg)
            print(result_msg)
            history.append(result_msg)

            print("Resposta:", result_msg.content)


# -------------------------------
# 3. Execução: criar agente e interagir
# -------------------------------
if __name__ == "__main__":
    # Inicializar modelo
    model = "gpt-4o-mini"
    chat_model = init_chat_model(model, model_provider="openai", temperature=0)

    # Inicializar armazenamento de grafo
    try:
        graph_store = MongoDBGraphStore.from_connection_string(
            connection_string=MONGO_URI,
            database_name=MONGO_DB_NAME,
            collection_name=MONGO_COLLECTION,
            entity_extraction_model=chat_model
        )
    except CollectionInvalid:
        graph_store = MongoDBGraphStore(
            connection_string=MONGO_URI,
            database_name=MONGO_DB_NAME,
            collection_name=MONGO_COLLECTION,
            entity_extraction_model=chat_model
        )

    # Criar retriever
    retriever = MongoDBGraphRAGRetriever(graph_store=graph_store)

    # Criar agente
    agent = ConversationalAgentManual(retriever, model=model)

    # Fazer perguntas
    questions = [
        "Quem foi Sherlock Holmes?",
        # "Qual era a profissão do Dr. Watson?",
        # "Onde Sherlock morava?"
    ]
    agent.response(questions)



Pergunta: Quem foi Sherlock Holmes?
interpret message content='Quem foi Sherlock Holmes?' additional_kwargs={} response_metadata={}
interpret response {'input': 'Quem foi Sherlock Holmes?', 'text': 'Sherlock Holmes é um personagem fictício criado pelo autor britânico Sir Arthur Conan Doyle. Ele é um detetive consultor conhecido por suas habilidades de observação, raciocínio lógico e dedução. Holmes apareceu pela primeira vez na novela "A Study in Scarlet" (Um Estudo em Vermelho), publicada em 1887. Ao longo dos anos, ele se tornou um dos personagens mais icônicos da literatura policial, aparecendo em várias histórias, incluindo "The Sign of the Four" (O Signo dos Quatro) e uma série de contos compilados em "The Adventures of Sherlock Holmes" (As Aventuras de Sherlock Holmes).\n\nHolmes é frequentemente acompanhado por seu amigo e narrador das histórias, Dr. John Watson. O personagem é conhecido por seu uso de métodos científicos e sua capacidade de resolver crimes aparentemente insolú

In [2]:
from typing import List
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.tools import StructuredTool
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_mongodb.retrievers.graphrag import MongoDBGraphRAGRetriever
from langchain_mongodb.graphrag.graph import MongoDBGraphStore
from pymongo.errors import CollectionInvalid
from langchain.chat_models import init_chat_model


# -------------------------------
# 1. Tool estruturada: consulta ao grafo
# -------------------------------
class QueryInput(BaseModel):
    query: str


class KnowledgeTool:
    def __init__(self, retriever: MongoDBGraphRAGRetriever):
        self.tool = StructuredTool.from_function(
            func=self._search,
            name="graph_search",
            description="Consulta o grafo com base em uma pergunta em linguagem natural.",
            args_schema=QueryInput
        )
        self.retriever = retriever

    def _search(self, query: str) -> str:
        docs = self.retriever.invoke(query)
        if not docs:
            return ""  # Retorna string vazia para detectar ausência
        return "\n\n".join([doc.page_content for doc in docs])


# -------------------------------
# 2. Agente conversacional sem LangGraph
# -------------------------------
class ConversationalAgentManual:
    def __init__(self, retriever: MongoDBGraphRAGRetriever, model="gpt-4o-mini", temperature=0):
        self.llm = ChatOpenAI(model=model, temperature=temperature)
        self.chain_with_context = LLMChain(
            llm=self.llm,
            prompt=ChatPromptTemplate.from_messages([
                ("system", "Você é um assistente que responde perguntas com base em dados estruturados de um grafo de conhecimento."),
                ("user", "Pergunta: {question}\n\nInformações do grafo:\n{context}")
            ])
        )
        self.chain_fallback = LLMChain(
            llm=self.llm,
            prompt=ChatPromptTemplate.from_messages([
                ("system", "Você é um assistente inteligente. Responda a pergunta abaixo com base em seu conhecimento."),
                ("user", "{question}")
            ])
        )
        self.knowledge_tool = KnowledgeTool(retriever)

    def graph_search(self, message: HumanMessage) -> str:
        print('graph_search: ', message)
        return self.knowledge_tool.tool.invoke({"query": message.content})

    def interpret(self, question: str, context: str) -> AIMessage:
        print('interpret question: ', question)
        print('interpret context: ', context)
        if context:
            response = self.chain_with_context.invoke({"question": question, "context": context})
        else:
            response = self.chain_fallback.invoke({"question": question})
        print('interpret response: ', response)
        return AIMessage(content=response["text"])

    def response(self, questions: List[str]):
        history: List[BaseMessage] = []

        for question in questions:
            print(f"\nPergunta: {question}")
            user_msg = HumanMessage(content=question)
            history.append(user_msg)

            # Etapa 1: buscar no grafo
            graph_info = self.graph_search(user_msg)
            print('graph_search response: ', graph_info)

            # Etapa 2: gerar resposta com ou sem contexto do grafo
            final_response = self.interpret(question=question, context=graph_info)
            print('final_response: ', final_response)
            history.append(final_response)

            print("Resposta:", final_response.content)


# -------------------------------
# 3. Execução
# -------------------------------
if __name__ == "__main__":
    # Inicializar modelo
    model = "gpt-4o-mini"
    chat_model = init_chat_model(model, model_provider="openai", temperature=0)

    # Inicializar armazenamento de grafo
    try:
        graph_store = MongoDBGraphStore.from_connection_string(
            connection_string=MONGO_URI,
            database_name=MONGO_DB_NAME,
            collection_name=MONGO_COLLECTION,
            entity_extraction_model=chat_model
        )
    except CollectionInvalid:
        graph_store = MongoDBGraphStore(
            connection_string=MONGO_URI,
            database_name=MONGO_DB_NAME,
            collection_name=MONGO_COLLECTION,
            entity_extraction_model=chat_model
        )

    # Criar retriever
    retriever = MongoDBGraphRAGRetriever(graph_store=graph_store)

    # Criar agente
    agent = ConversationalAgentManual(retriever, model=model)

    # Perguntas de exemplo
    questions = [
        "Quem foi Sherlock Holmes?",
        # "Qual é o nome do cachorro do Batman?",  # Pergunta provavelmente sem grafo, ativará fallback
    ]
    agent.response(questions)


/tmp/ipykernel_16412/322164799.py:44: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain_with_context = LLMChain(



Pergunta: Quem foi Sherlock Holmes?
graph_search:  content='Quem foi Sherlock Holmes?' additional_kwargs={} response_metadata={}


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from typing import List, TypedDict
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langgraph.graph import StateGraph
from langchain.tools import StructuredTool
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.graphs.graph_document import GraphDocument
from langchain.chat_models import init_chat_model


# -------------------------------
# 1. Estado do grafo
# -------------------------------
class MyState(TypedDict):
    messages: List[BaseMessage]


# -------------------------------
# 2. Tool estruturada: consulta ao grafo
# -------------------------------
class QueryInput(BaseModel):
    consulta: str


class KnowledgeTool:
    def __init__(self, retriever):
        self.tool = StructuredTool.from_function(
            func=self._search,
            name="graph_search",
            description="Consulta o grafo com base em uma question em linguagem natural.",
            args_schema=QueryInput
        )
        self.retriever = retriever

    def _search(self, consulta: str) -> str:
        """
        Consulta um grafo de conhecimento com uma string de busca.
        """
        docs = self.retriever.invoke(consulta)
        if not docs:
            return "Nenhum resultado encontrado no grafo."
        return "\n\n".join([doc.page_content for doc in docs])

    def run(self, state: MyState) -> MyState:
        last_msg = state["messages"][-1]
        if not isinstance(last_msg, AIMessage):
            raise ValueError("Esperava-se uma AIMessage da etapa anterior.")

        resultado = self.tool.invoke({"consulta": last_msg.content})
        nova = AIMessage(content=resultado)
        return {"messages": state["messages"] + [nova]}


# -------------------------------
# 3. Agente conversacional com grafo
# -------------------------------
class ConversationalAgent:
    def __init__(self, retriever, model="gpt-4o-mini", temperature=0):
        self.llm = ChatOpenAI(model=model, temperature=temperature)
        self.chain = LLMChain(
            llm=self.llm,
            prompt=ChatPromptTemplate.from_messages([
                ("system", "Você é um assistente que responde questions com base em um grafo de conhecimento."),
                ("user", "{input}")
            ])
        )
        self.knowledge_tool = KnowledgeTool(retriever)
        self.graph = self._graph_create()

    def _interpret(self, state: MyState) -> MyState:
        """
        Usa o modelo LLM para reescrever ou interpretar a question do usuário.
        """
        last_msg = state["messages"][-1]
        if not isinstance(last_msg, HumanMessage):
            raise ValueError("Esperava-se uma HumanMessage.")
        
        response = self.chain.invoke({"input": last_msg.content})
        nova = AIMessage(content=response["text"])
        return {"messages": state["messages"] + [nova]}

    def _format_response(self, state: MyState) -> MyState:
        """
        Retorna apenas a última mensagem como resposta final.
        """
        return {"messages": [state["messages"][-1]]}

    def _graph_create(self):
        workflow = StateGraph(state_schema=MyState)
        workflow.add_node("interpret", self._interpret)
        workflow.add_node("graph_search", self.knowledge_tool.run)
        workflow.add_node("format_response", self._format_response)

        workflow.set_entry_point("interpret")
        workflow.add_edge("interpret", "format_response")

        return workflow.compile()

    def response(self, questions: List[str]):
        """
        Processa múltiplas perguntas mantendo o histórico da conversa.
        """
        hystory: List[BaseMessage] = []

        for question in questions:
            print(f"\nPergunta: {question}")
            hystory.append(HumanMessage(content=question))
            state = {"messages": hystory}

            response = self.graph.invoke(state)
            response_msg = response["messages"][-1]
            hystory.append(response_msg)

            print("Resposta:", response_msg.content)


# -------------------------------
# 4. Execução: criar agente e interagir
# -------------------------------
if __name__ == "__main__":
    # model = "gpt-4o"
    model="gpt-4o-mini"
    chat_model = init_chat_model(model, model_provider="openai", temperature=0)

    # 1. Gerar os documentos do grafo a partir de uma transformação LLM
    graph_transformer = LLMGraphTransformer(llm=chat_model)
    
    # Exemplo de dados para transformação em GraphDocument
    raw_documents = [
        {"title": "Sherlock Holmes", "content": "Sherlock Holmes é um detetive fictício criado por Arthur Conan Doyle."},
        {"title": "Dr. Watson", "content": "Dr. Watson era o fiel companheiro de Sherlock Holmes."}
    ]
    
    print(raw_documents)
    # Transformar os dados brutos em GraphDocuments
    graph_documents = [GraphDocument.from_dict(doc) for doc in raw_documents]
    print(graph_documents)
    
    # 2. Criar o retriever para usar esses documentos no agente
    retriever = graph_transformer.create_graph_retriever(graph_documents)
    
    # 3. Criar o agente
    # agent = ConversationalAgent(retriever, model="gpt-4o-mini")
    # questions = [
    #     "Quem foi Sherlock Holmes?",
    #     # "Qual era a profissão do Dr. Watson?"
    # ]
    # agent.response(questions)


[{'title': 'Sherlock Holmes', 'content': 'Sherlock Holmes é um detetive fictício criado por Arthur Conan Doyle.'}, {'title': 'Dr. Watson', 'content': 'Dr. Watson era o fiel companheiro de Sherlock Holmes.'}]


AttributeError: from_dict